<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_2/03_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習
Music VAEを使って、自由に作曲しましょう。  
主に、 曲の最初のNoteSequenceと、最後のNoteSequenceに変更を加えます。  

## ライブラリのインストール

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

## チェックポイントのダウンロード

In [ ]:
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

## モデルの初期化

In [ ]:
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

# モデルの初期化
music_vae = TrainedModel(
      configs.CONFIG_MAP["cat-mel_2bar_big"], 
      batch_size=4,  # 一度に処理するデータ数
      checkpoint_dir_or_path="/content/mel_2bar_big.ckpt")

## NoteSequenceの生成

温度（ランダム度合い）を、好みに合わせて変更しましょう。    

In [ ]:
import note_seq

generated = music_vae.sample(n=5,  # 生成数
                             length=128,  # ステップ数
                             temperature=1.0)  # 温度

for ns in generated:
    note_seq.plot_sequence(ns)
    note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

## 最初と最後のNoteSequence

曲の最初のNoteSequenceと、最後のNoteSequenceに変更を加えましょう。


In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

# 最初のNoteSeqence
kira2_start = music_pb2.NoteSequence()

kira2_start.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
kira2_start.notes.add(pitch=60, start_time=0.4, end_time=0.8, velocity=80)
kira2_start.notes.add(pitch=67, start_time=0.8, end_time=1.2, velocity=80)
kira2_start.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
kira2_start.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
kira2_start.notes.add(pitch=69, start_time=2.0, end_time=2.4, velocity=80)
kira2_start.notes.add(pitch=67, start_time=2.4, end_time=3.2, velocity=80)
kira2_start.notes.add(pitch=65, start_time=3.2, end_time=3.6, velocity=80)
kira2_start.notes.add(pitch=65, start_time=3.6, end_time=4.0, velocity=80)
kira2_start.notes.add(pitch=64, start_time=4.0, end_time=4.4, velocity=80)
kira2_start.notes.add(pitch=64, start_time=4.4, end_time=4.8, velocity=80)
kira2_start.notes.add(pitch=62, start_time=4.8, end_time=5.2, velocity=80)
kira2_start.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
kira2_start.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

kira2_start.total_time = 6.4 
kira2_start.tempos.add(qpm=75);

note_seq.plot_sequence(kira2_start)
note_seq.play_sequence(kira2_start, synth=note_seq.fluidsynth)

# 最後のNoteSeqence
kira2_end = music_pb2.NoteSequence()

kira2_end.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
kira2_end.notes.add(pitch=62, start_time=0.4, end_time=0.8, velocity=80)
kira2_end.notes.add(pitch=64, start_time=0.8, end_time=1.2, velocity=80)
kira2_end.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
kira2_end.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
kira2_end.notes.add(pitch=64, start_time=2.0, end_time=2.4, velocity=80)
kira2_end.notes.add(pitch=60, start_time=2.4, end_time=3.2, velocity=80)
kira2_end.notes.add(pitch=62, start_time=3.2, end_time=3.6, velocity=80)
kira2_end.notes.add(pitch=64, start_time=3.6, end_time=4.0, velocity=80)
kira2_end.notes.add(pitch=67, start_time=4.0, end_time=4.4, velocity=80)
kira2_end.notes.add(pitch=69, start_time=4.4, end_time=4.8, velocity=80)
kira2_end.notes.add(pitch=64, start_time=4.8, end_time=5.2, velocity=80)
kira2_end.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
kira2_end.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

kira2_end.total_time = 6.4
kira2_end.tempos.add(qpm=75); 

note_seq.plot_sequence(kira2_end)
note_seq.play_sequence(kira2_end, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## NoteSequnce間の補間
どのような曲が生成されるのか、以下のコードにより確認します。  

In [ ]:
n_seq = 8  # 曲のNoteSeqence数（最初と最後を含む）

# NoteSeqenceを複数生成し、リストに格納
gen_seq = music_vae.interpolate(
    kira2_start,  # 最初のNoteSeqence
    kira2_end,  # 最後のNoteSeqence
    num_steps=n_seq,
    length=32)

# NoteSeqenceを全て結合し、1つの曲に
interp_seq = note_seq.sequences_lib.concatenate_sequences(gen_seq)

note_seq.plot_sequence(interp_seq)
note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)

## MIDIファイルの保存とダウンロード

In [ ]:
from google.colab import files

note_seq.sequence_proto_to_midi_file(interp_seq, "exercise.mid")  #MIDI　データに変換し保存
files.download("exercise.mid")  # ダウンロード